**CONTAR CANTIDAD IMAGENES EN MURAV2.1**

In [1]:
import os

# Ruta de la carpeta de datos
base_drive_path = "G:/Mi unidad/TESIS/"  # Asegurar que esta ruta es correcta
data_directory = os.path.join(base_drive_path, "MURA_v2.1")

def count_images_per_category(directory):
    category_counts = {}

    for split in ["train", "valid"]:
        split_path = os.path.join(directory, split)
        if not os.path.exists(split_path):
            print(f"⚠️ No se encontró la carpeta: {split_path}")
            continue

        category_counts[split] = {}
        for category in os.listdir(split_path):
            category_path = os.path.join(split_path, category)
            if os.path.isdir(category_path):
                total_images = 0
                for root, _, files in os.walk(category_path):  # Recorre todas las subcarpetas
                    total_images += sum(1 for file in files if file.endswith(('.png', '.jpg', '.jpeg')))
                category_counts[split][category] = total_images

    return category_counts

# Contar imágenes en train y valid
category_image_counts = count_images_per_category(data_directory)

# Mostrar resultados
for split, categories in category_image_counts.items():
    print(f"\n📂 {split.upper()} Dataset")
    for category, count in categories.items():
        print(f"   {category}: {count} imágenes")



📂 TRAIN Dataset
   XR_FOREARM: 1825 imágenes
   XR_WRIST: 9752 imágenes
   XR_ELBOW: 4931 imágenes
   XR_SHOULDER: 8379 imágenes
   XR_HUMERUS: 1272 imágenes
   XR_HAND: 5543 imágenes
   XR_FINGER: 5106 imágenes

📂 VALID Dataset
   XR_FOREARM: 301 imágenes
   XR_FINGER: 461 imágenes
   XR_SHOULDER: 563 imágenes
   XR_HAND: 460 imágenes
   XR_HUMERUS: 288 imágenes
   XR_ELBOW: 465 imágenes
   XR_WRIST: 659 imágenes


**APLICACION DE CLAHE Y UNSHARP MASK**

In [10]:
import os
import numpy as np
import time
from skimage import io, color, exposure, transform
from skimage.filters import unsharp_mask
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor

# Ajustar la ruta de Google Drive en Windows
base_drive_path = "G:/Mi unidad/TESIS"  # Asegurar que esta ruta es correcta en tu sistema

def apply_unsharp_mask(image, radius=3, amount=3):
    """ Aplica Unsharp Mask usando skimage para mejorar la nitidez """
    return unsharp_mask(image, radius=radius, amount=amount)

def apply_clahe(image, clip_limit=0.03):
    """ Aplica CLAHE usando skimage para mejorar el contraste """
    return exposure.equalize_adapthist(image, clip_limit=clip_limit)

def process_image(img_path, save_path):
    """ Carga, mejora y guarda una imagen válida """
    try:
        image = io.imread(img_path)
    except Exception as e:
        print(f"❌ No se pudo cargar: {img_path} - Error: {e}")
        return False
    
    if image is None or image.size == 0:
        print(f"❌ Imagen corrupta o vacía: {img_path}")
        return False
    
    # Convertir a escala de grises si es RGB
    if len(image.shape) == 3:
        image = color.rgb2gray(image)

    # Redimensionar la imagen 
    image = transform.resize(image, (384, 384), anti_aliasing=True)

    # Aplicar filtros
    enhanced_image = apply_unsharp_mask(image)
    enhanced_image = apply_clahe(enhanced_image)

    # Guardar imagen mejorada
    io.imsave(save_path, (enhanced_image * 255).astype(np.uint8))
    return True

def process_images_parallel(input_folder, output_folder, num_workers=8):
    """ Procesa imágenes en paralelo recorriendo todas las subcarpetas, excluyendo imágenes corruptas """
    os.makedirs(output_folder, exist_ok=True)

    image_paths = []
    failed_images = []
    
    # Recorrer todas las subcarpetas para encontrar imágenes
    for root, _, files in os.walk(input_folder):
        for file in files:
            if file.endswith(('.png', '.jpg', '.jpeg')):
                relative_path = os.path.relpath(root, input_folder)
                save_path = os.path.join(output_folder, relative_path)
                os.makedirs(save_path, exist_ok=True)
                image_paths.append((os.path.join(root, file), os.path.join(save_path, file)))

    if not image_paths:
        print("⚠️ No se encontraron imágenes. Verifica la estructura de MURA-v1.1.")
        return

    # Procesamiento en paralelo
    start_time = time.time()
    print(f" Procesando {len(image_paths)} imágenes con {num_workers} hilos...")

    with tqdm(total=len(image_paths), desc="Procesando imágenes", unit="img", dynamic_ncols=True) as pbar:
        with ThreadPoolExecutor(max_workers=num_workers) as executor:
            results = list(executor.map(lambda p: process_image(*p), image_paths))
            pbar.update(len(results))

    # Registrar imágenes fallidas
    failed_images = [image_paths[i][0] for i, success in enumerate(results) if not success]
    print(f"\n Procesamiento completado en {time.time() - start_time:.2f} segundos.")
    print(f" Total de imágenes procesadas: {len(image_paths) - len(failed_images)}")
    print(f" Total de imágenes fallidas: {len(failed_images)}")
    
    if failed_images:
        print("❌ Imágenes corruptas que no fueron agregadas:")
        for img in failed_images:
            print(img)

# Directorios de entrada y salida en Google Drive en Windows
input_directory = os.path.join(base_drive_path, "MURA-v1.1")
output_directory = os.path.join(base_drive_path, "MURA_UC")  # Nueva carpeta para guardar imágenes procesadas

# Procesar imágenes en paralelo con 8 hilos
tqdm.write(" Iniciando el procesamiento de imágenes...")
process_images_parallel(input_directory, output_directory, num_workers=8)
tqdm.write(" Procesamiento completado!")


 Iniciando el procesamiento de imágenes...
 Procesando 40009 imágenes con 8 hilos...


Procesando imágenes:   0%|                                                                  | 0/40009 [00:00<?, ?img/s]

❌ No se pudo cargar: G:/Mi unidad/TESIS\MURA-v1.1\train\XR_WRIST\patient07840\study2_negative\._image3.png - Error: Could not find a backend to open `G:/Mi unidad/TESIS\MURA-v1.1\train\XR_WRIST\patient07840\study2_negative\._image3.png`` with iomode `r`.
Based on the extension, the following plugins might add capable backends:
  pyav:  pip install imageio[pyav]
❌ No se pudo cargar: G:/Mi unidad/TESIS\MURA-v1.1\train\XR_WRIST\patient07840\study1_negative\._image1.png - Error: Could not find a backend to open `G:/Mi unidad/TESIS\MURA-v1.1\train\XR_WRIST\patient07840\study1_negative\._image1.png`` with iomode `r`.
Based on the extension, the following plugins might add capable backends:
  pyav:  pip install imageio[pyav]
❌ No se pudo cargar: G:/Mi unidad/TESIS\MURA-v1.1\train\XR_WRIST\patient07840\study2_negative\._image1.png - Error: Could not find a backend to open `G:/Mi unidad/TESIS\MURA-v1.1\train\XR_WRIST\patient07840\study2_negative\._image1.png`` with iomode `r`.
Based on the exte

C:\Users\SKACE\AppData\Local\Temp\ipykernel_11716\3870136753.py:44: UserWarning: G:/Mi unidad/TESIS\MURA_UC\train\XR_HAND\patient10593\study1_negative\image2.png is a low contrast image
  io.imsave(save_path, (enhanced_image * 255).astype(np.uint8))
C:\Users\SKACE\AppData\Local\Temp\ipykernel_11716\3870136753.py:44: UserWarning: G:/Mi unidad/TESIS\MURA_UC\train\XR_FINGER\patient04687\study1_negative\image2.png is a low contrast image
  io.imsave(save_path, (enhanced_image * 255).astype(np.uint8))
Procesando imágenes: 100%|██████████████████████████████████████████████████████| 40009/40009 [11:37<00:00, 57.33img/s]


 Procesamiento completado en 697.92 segundos.
 Total de imágenes procesadas: 40005
 Total de imágenes fallidas: 4
❌ Imágenes corruptas que no fueron agregadas:
G:/Mi unidad/TESIS\MURA-v1.1\train\XR_WRIST\patient07840\study1_negative\._image1.png
G:/Mi unidad/TESIS\MURA-v1.1\train\XR_WRIST\patient07840\study2_negative\._image2.png
G:/Mi unidad/TESIS\MURA-v1.1\train\XR_WRIST\patient07840\study2_negative\._image3.png
G:/Mi unidad/TESIS\MURA-v1.1\train\XR_WRIST\patient07840\study2_negative\._image1.png
 Procesamiento completado!


**DIVIDIR 80 Y 20% DATOS**

In [1]:
import os
import shutil
import random
import pandas as pd

# Ruta del dataset original (usaremos tanto train como valid)
ruta_base = "G:/Mi unidad/TESIS/MURA_UC"
ruta_original_train = os.path.join(ruta_base, "train")
ruta_original_valid = os.path.join(ruta_base, "valid")  # Se incluirá en la división

# Nueva carpeta donde se almacenará la división
ruta_nueva_base = "G:/Mi unidad/TESIS/MURA_UCD"
ruta_nueva_train = os.path.join(ruta_nueva_base, "train")
ruta_nueva_valid = os.path.join(ruta_nueva_base, "valid")

# Crear las nuevas carpetas si no existen
os.makedirs(ruta_nueva_train, exist_ok=True)
os.makedirs(ruta_nueva_valid, exist_ok=True)

# Listas para almacenar las rutas y etiquetas
datos = []  # Lista general para las imágenes

# Función para recopilar imágenes de ambas carpetas
def recopilar_imagenes(ruta_base):
    imagenes = []
    if os.path.isdir(ruta_base):
        for tipo in os.listdir(ruta_base):  # Recorrer XR_ELBOW, XR_FINGER, etc.
            ruta_tipo = os.path.join(ruta_base, tipo)
            if os.path.isdir(ruta_tipo):
                for paciente in os.listdir(ruta_tipo):  # Recorrer patient0001, patient0002, etc.
                    ruta_paciente = os.path.join(ruta_tipo, paciente)
                    if os.path.isdir(ruta_paciente):
                        for estudio in os.listdir(ruta_paciente):  # Recorrer study1_negative, study2_positive, etc.
                            ruta_estudio = os.path.join(ruta_paciente, estudio)
                            if os.path.isdir(ruta_estudio):
                                for img in os.listdir(ruta_estudio):  # Listar imágenes
                                    img_path = os.path.join(ruta_estudio, img)
                                    etiqueta = 0 if "negative" in estudio.lower() else 1  # Etiqueta 0 o 1
                                    imagenes.append((img_path, etiqueta, tipo, paciente, estudio))
    return imagenes

# Recopilar imágenes de ambas carpetas
imagenes_total = recopilar_imagenes(ruta_original_train) + recopilar_imagenes(ruta_original_valid)

# Mezclar imágenes aleatoriamente
random.shuffle(imagenes_total)

# Calcular la cantidad exacta para train y valid
num_train = int(len(imagenes_total) * 0.8)
imagenes_train = imagenes_total[:num_train]
imagenes_valid = imagenes_total[num_train:]

# Copiar imágenes y crear CSV
datos_train, datos_valid = [], []

for img_path, etiqueta, tipo, paciente, estudio in imagenes_train:
    ruta_tipo_train = os.path.join(ruta_nueva_train, tipo)
    ruta_paciente_train = os.path.join(ruta_tipo_train, paciente)
    ruta_estudio_train = os.path.join(ruta_paciente_train, estudio)
    os.makedirs(ruta_estudio_train, exist_ok=True)
    nueva_ruta = os.path.join(ruta_estudio_train, os.path.basename(img_path))

    try:
        shutil.copy(img_path, nueva_ruta)
        datos_train.append([nueva_ruta, etiqueta])
    except Exception as e:
        print(f"Error al copiar {img_path}: {e}")

for img_path, etiqueta, tipo, paciente, estudio in imagenes_valid:
    ruta_tipo_valid = os.path.join(ruta_nueva_valid, tipo)
    ruta_paciente_valid = os.path.join(ruta_tipo_valid, paciente)
    ruta_estudio_valid = os.path.join(ruta_paciente_valid, estudio)
    os.makedirs(ruta_estudio_valid, exist_ok=True)
    nueva_ruta = os.path.join(ruta_estudio_valid, os.path.basename(img_path))

    try:
        shutil.copy(img_path, nueva_ruta)
        datos_valid.append([nueva_ruta, etiqueta])
    except Exception as e:
        print(f"Error al copiar {img_path}: {e}")

# Guardar los CSV
df_train = pd.DataFrame(datos_train, columns=["ruta", "etiqueta"])
df_valid = pd.DataFrame(datos_valid, columns=["ruta", "etiqueta"])

df_train.to_csv(os.path.join(ruta_nueva_base, "train.csv"), index=False)
df_valid.to_csv(os.path.join(ruta_nueva_base, "valid.csv"), index=False)

print(f"✅ División completada. CSVs generados en {ruta_nueva_base}")


✅ División completada. CSVs generados en G:/Mi unidad/TESIS/MURA_UCD


**CONTAR CANRIDAD DE IMAGENES DESPUES DE DIVISION**

In [2]:
import os

# Ruta a la carpeta nueva donde se dividieron las imágenes
ruta_nueva_base = "G:/Mi unidad/TESIS/MURA_UCD"
ruta_train = os.path.join(ruta_nueva_base, "train")
ruta_valid = os.path.join(ruta_nueva_base, "valid")

# Función para contar imágenes en una carpeta
def contar_imagenes(ruta_base):
    total_imagenes = 0
    for tipo in os.listdir(ruta_base):  # Recorrer XR_ELBOW, XR_FINGER, etc.
        ruta_tipo = os.path.join(ruta_base, tipo)
        if os.path.isdir(ruta_tipo):
            for paciente in os.listdir(ruta_tipo):  # Recorrer patient0001, patient0002, etc.
                ruta_paciente = os.path.join(ruta_tipo, paciente)
                if os.path.isdir(ruta_paciente):
                    for estudio in os.listdir(ruta_paciente):  # Recorrer study1_negative, study2_positive, etc.
                        ruta_estudio = os.path.join(ruta_paciente, estudio)
                        if os.path.isdir(ruta_estudio):
                            total_imagenes += len(os.listdir(ruta_estudio))  # Contar archivos en cada estudio
    return total_imagenes

# Contar imágenes en train y valid
total_train = contar_imagenes(ruta_train)
total_valid = contar_imagenes(ruta_valid)

# Mostrar resultados
print(f"Total de imágenes en TRAIN: {total_train}")
print(f"Total de imágenes en VALID: {total_valid}")

# Verificar que la suma sea 40,005
total_general = total_train + total_valid
print(f"Total de imágenes en el dataset dividido: {total_general}")


Total de imágenes en TRAIN: 32004
Total de imágenes en VALID: 8001
Total de imágenes en el dataset dividido: 40005
